In [130]:
from sklearn.datasets import make_friedman3, make_regression, make_friedman2, fetch_california_housing
from sklearn.metrics import mean_squared_error
import numpy as np
import random
np.random.seed(8)
random.seed(8)

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
# data = fetch_california_housing()

In [133]:
# X, y = data.data, data.target
# X, y = X[:500], y[:500]
X, y = make_friedman2(n_samples=1000, noise=0.1)
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
import numpy as np

In [135]:
%%time
from lightgbm import LGBMRegressor

gb_regressor = LGBMRegressor()

# Fit the model to the training data
gb_regressor.fit(X_train, y_train)

print(f"Mean Squared Error on Validation Set: {mean_squared_error(gb_regressor.predict(X_test), y_test)}")
print(f"Mean Squared Error on Train Set: {mean_squared_error(gb_regressor.predict(X_train), y_train)}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000085 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1000
[LightGBM] [Info] Number of data points in the train set: 750, number of used features: 4
[LightGBM] [Info] Start training from score 465.609118
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [136]:
%%time
from xgboost import XGBRegressor
gb_regressor = XGBRegressor()

# Fit the model to the training data
gb_regressor.fit(X_train, y_train)

print(f"Mean Squared Error on Validation Set: {mean_squared_error(gb_regressor.predict(X_test), y_test)}")
print(f"Mean Squared Error on Train Set: {mean_squared_error(gb_regressor.predict(X_train), y_train)}")

Mean Squared Error on Validation Set: 393.91439519311524
Mean Squared Error on Train Set: 1.0886295737388614
CPU times: user 1.96 s, sys: 7.55 ms, total: 1.97 s
Wall time: 298 ms


In [146]:
import numpy as np
import pandas as pd
from collections import Counter
from pandas import DataFrame, concat
from concurrent.futures import ThreadPoolExecutor
# Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.svm import NuSVR, SVC, SVR, LinearSVR
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge, LassoCV, RidgeCV, LarsCV, OrthogonalMatchingPursuitCV, LassoLarsCV, ElasticNet, ElasticNetCV, SGDRegressor, LassoLars, Lasso, Ridge, ARDRegression, RANSACRegressor, HuberRegressor, TheilSenRegressor, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from time import perf_counter
from random import sample
from copy import deepcopy

class MSBoostRegressor(BaseEstimator, RegressorMixin):
    """A Variational Gradient Boosting Regressor
    """

    def __init__(self):
        """ Initialize VGBRegressor Object
        """

    def _metrics(self, vt, vp, model, time=None):
        """get loss metrics of a model

        Args:
            vt (iterable): validation true values
            vp (iterable): validation pred values
            model (object): any model with fit and predict method
            time (float, optional): execution time of the model. Defaults to None.

        Returns:
            dict['model', 'time', 'loss']
        """
        if self.custom_loss_metrics:
            return {'model': model, 'time': time, 'loss': self.custom_loss_metrics(vt, vp)}
        return {"model": model, "time": time, "loss": mean_squared_error(vt, vp)}

    def _create_model(self, X, y, model_name, time_it: bool = False):
        """fit a model instance

        Args:
            X (iterable)
            y (iterable)
            model_name (object): any model object with fit and predict methods
            time_it (bool, optional): measure execution time. Defaults to False.

        Returns:
            tuple(model, time=None)
        """
        model = model_name()
        if time_it:
            begin = perf_counter()
            model.fit(X, y)
            end = perf_counter()
            return (model, end - begin)
        return (model.fit(X, y), None)

    def _get_metrics(self, model_name):
        """a helper fuction, combines self._create_model and self._metrics

        Args:
            model_name (object): any model with fit and predict methods

        Returns:
            self._metrics
        """
        try:
            Xt, Xv, yt, yv = train_test_split(self._X, self._y)
            results = self._create_model(Xt, yt, model_name, time_it=False)
            model, time = results[0], results[1]
            return self._metrics(yv,
                                 model.predict(Xv), model, time)
        except Exception:
            return None

    def _get_results(self, X, y) -> list:
        """Use multi-threading to return all results

        Args:
            X (iterable)
            y (iterable)

        Returns:
            list[dict['model', 'time', 'loss']]
        """
        results = []
        # self._X = self._minimax.fit_transform(self._robust.fit_transform(
        #         KNNImputer(weights='distance').fit_transform(X)))
        self._X = X
        self._y = y
        with ThreadPoolExecutor(max_workers=len(self._models)) as executor:
            res = executor.map(self._get_metrics, self._models)
            results = [i for i in res if i]
        return results

    def fit(
        self, X, y,
        early_stopping: bool = False,
        early_stopping_min_delta: float = 0.001,
        early_stopping_patience: int = 10,
        custom_models: list = None,
        learning_rate: float = 0.1,
        n_estimators: int = 100,
        warm_start: bool = False,
        complexity: bool = False,
        light: bool = False,
        custom_loss_metrics: object = False,
        freeze_models: bool = False,
        n_models: int = 5,
        n_iter_models: int = 5,
        n_warm: int = None,
        n_random_models: int = 6,
        return_vals: bool = True,
        stacking_model=LassoLarsCV,
    ):
        """fit VGBoost model

        Args:
            X (iterable)
            y (iterbale)
            early_stopping (bool, optional): Defaults to False.
            early_stopping_min_delta (float, optional): Defaults to 0.001.
            early_stopping_patience (int, optional): Defaults to 10.
            custom_models (tuple, optional): tuple of custom models with fit and predict methods. Defaults to None.
            learning_rate (float, optional): Defaults to 0.05.
            n_estimators (int, optional): Defaults to 100.
            warm_start (bool, optional): Defaults to False.
            complexity (bool, optional): trains more models but has greater time complexity. Defaults to False.
            light (bool, optional): trains less models. Defaults to True.
            custom_loss_metrics (object, optional): _description_. Defaults to False.
            freeze_models (bool, optional): test only a selected models. Defaults to False.
            n_models (int, optional): Applicable for freeze_models, number of models to train. Defaults to 5.
            n_iter_models (int, optional): Applicable for freeze_models, number of iterations before finalizing the models. Defaults to 5.
            n_warm (int, optional): Applicable for warm start, number of iterarions to store. Defaults to None.
            n_random_models (int, optional): train on a random number of models. Defaults to 0.
            return_vals (bool, optional): returns analytics. Defaults to True.

        Returns:
            tuple[final ensemble sequence, mean absolute error of each layer, residual value of each layer],
            None
        """
        X, y = check_X_y(X, y)
        self.classes_ = np.array(set(y))
        self.stacking_model = stacking_model
        self.y_max = max(y)
        # self.n_classes_ = len(self.classes_)
        self.len_X = X.shape[0]
        self.n_features_in_ = X.shape[1]
        if custom_models:
            self._models = custom_models
        self.custom_loss_metrics = custom_loss_metrics
        self.learning_rate = learning_rate
        # self.final_estimator = final_estimator
        self.n_estimators = n_estimators
        self.early_stopping = early_stopping
        self.early_stopping_min_delta = early_stopping_min_delta
        self.early_stopping_patience = early_stopping_patience
        if custom_models:

            self._models_lst = custom_models
        else:
            if complexity:
                self._models_lst = (DecisionTreeRegressor, LinearRegression, BayesianRidge, KNeighborsRegressor, HistGradientBoostingRegressor, LGBMRegressor, GradientBoostingRegressor, XGBRegressor,
                                    ElasticNetCV, LassoLarsCV, LassoCV, ExtraTreesRegressor, SVC, 
                                    BaggingRegressor, NuSVR, SGDRegressor, KernelRidge, MLPRegressor,
                                    RidgeCV, ARDRegression, RANSACRegressor, HuberRegressor, TheilSenRegressor, LassoLarsIC)
            elif light:
                self._models_lst = (LGBMRegressor, ExtraTreesRegressor,
                                    BaggingRegressor, RANSACRegressor, LassoLarsIC, BayesianRidge)
            else:
                self._models_lst = (DecisionTreeRegressor, LinearRegression, BayesianRidge, KNeighborsRegressor, LGBMRegressor,
                                    ElasticNet, LassoLars, Lasso, SGDRegressor, BaggingRegressor, ExtraTreesRegressor,
                                    Ridge, ARDRegression, RANSACRegressor, LassoLarsIC)
            self._models = deepcopy(self._models_lst)
        self.freeze_models = freeze_models
        if self.freeze_models:
            self.n_models = n_models
            self.n_iter_models = n_iter_models
        self._y_mean = 0
        # base model: mean
        # computer residuals: y - y hat
        # for n_estimators: a) y = prev residuals && residuals * learning rate
        # add early stopping
        # restore best weights
        # ada boost and adaptive scaling for learning rates
        self._ensemble = []
        preds = DataFrame(
            data={'yt': y, 'p0': np.full((len(y)), self._y_mean)})
        residuals = DataFrame(
            data={'r0': y - self._y_mean})
        errors = []
        if not early_stopping:
            if warm_start:
                # for i in range(1, self.n_estimators + 1):
                #     try:
                #         y = residuals[f'r{i - 1}']
                #     except KeyError:
                #         return residuals
                #     results = self._get_results(X, y)
                #     min_loss = min(results, key=lambda x: x.get(
                #         "loss", float('inf')))["loss"]  # https://stackoverflow.com/a/19619294
                #     min_model = [i['model']
                #                  for i in results if min_loss >= i['loss']][0]
                #     preds[f'p{i}'] = residuals.sum(axis=1) + min_model.predict(
                #         X) * self.learning_rate
                #     residuals[f'r{i}'] = preds['yt'] - preds[f'p{i}']
                #     if i % n_warm == 0:
                #         X[f"r{i}"] = residuals[f'r{i}'].copy()
                #     try:
                #         errors.append(mean_squared_error(
                #             preds['yt'], preds[f'p{i}']))
                #     except Exception:
                #         df = concat(
                #             [preds['yt'], preds[f'p{i - 1}']], axis=1).dropna()
                #         errors.append(mean_squared_error(
                #             df['yt'], df[f"p{i - 1}"]))
                #     self._ensemble.append(min_model)
                pass
            else:
                freeze_models_lst = []
                for i in range(1, self.n_estimators + 1):
                    try:
                        y = residuals[f'r{i - 1}']
                    except KeyError:
                        return residuals, i, self._ensemble
                    results = self._get_results(X, y)
                    if n_random_models > 0:
                        self._models = tuple(
                            sample(self._models_lst, n_random_models))
                    elif self.freeze_models:
                        if self.n_iter_models > -1:
                            freeze_models_lst.append([i.get("model") for i in sorted(results, key=lambda x: x.get(
                                "loss", float('inf')))][:n_models])
                            self.n_iter_models -= 1
                        else:
                            model_lst = sorted(dict(Counter(i for sub in freeze_models_lst for i in set(
                                sub))).items(), key=lambda ele: ele[1], reverse=True)
                            # return model_lst
                            self._models = tuple(type(i[0]) for i in model_lst)[
                                :n_models]
                            # return self._models
                    try:
                        min_loss = min(results, key=lambda x: x.get(
                            "loss", float('inf')))["loss"]  # https://stackoverflow.com/a/19619294
                    except Exception:
                        return results
                    # min_model = [i['model']
                    #              for i in results if min_loss >= i['loss']][0]
                    
                    models = [i['model'] for i in results]
                    stacking_preds = {}
                    for j, model in enumerate(models):
                        # Store the predictions in the dictionary with a key based on the model index
                        stacking_preds[f'p{j}'] = model.predict(X)

                    stacking_estimator = stacking_model()
                    stacking_preds = pd.DataFrame(stacking_preds)
                    try:
                        stacking_estimator.fit(stacking_preds, y)
                    except ValueError:
                        stacking_estimator.fit(stacking_preds, y)
                        return stacking_preds, y
                    preds[f'p{i}'] = residuals.sum(axis=1) + stacking_estimator.predict(stacking_preds) * self.learning_rate
                    residuals[f'r{i}'] = preds['yt'] - preds[f'p{i}']
                    errors.append(mean_squared_error(
                        preds['yt'], preds[f'p{i}']))
                    models = [models]
                    models.extend([[stacking_estimator]])
                    self._ensemble.append(models)
                    if errors[i - 1] == 0:
                        break
        else:
            return self
        min_error = min(errors)
        min_error_i = [i for i in range(
            len(errors)) if errors[i] == min_error][0]
        self._ensemble, errors = self._ensemble[:
                                                min_error_i], errors[:min_error_i]
        residuals = residuals[:len(errors)]
        # preds = preds[preds.columns[:min_error_i + 2]]
        if return_vals:
            self.residuls = residuals
            self.errors = errors
            self.ensemble = self._ensemble
        # X, y = preds.drop("yt", axis=1), preds["yt"]
        # self.preds = X
        # self.final_estimator.fit(X, y)

    def predict(self, X_test):
        """
        Args:
            X_test (iterable)

        Returns:
            numpy.array: predictions
        """
        # check_is_fitted(self)
        X_test = check_array(X_test)
        # X_test = self._robust.transform(self._minimax.transform(deepcopy(X_test)))
        preds = DataFrame(
            data={'p0': np.full((len(X_test)), self._y_mean)})
        for i in range(len(self._ensemble)):
            models = self._ensemble[i][0]
            stacking_estimator = self._ensemble[i][1][0]
            stacking_preds = {}
            for j, model in enumerate(models):
                # Store the predictions in the dictionary with a key based on the model index
                stacking_preds[f'p{j}'] = model.predict(X_test)
            preds[f"p{i +1}"] = stacking_estimator.predict(pd.DataFrame(stacking_preds))
        # self.final_p = preds
        preds_ = preds.sum(axis=1)
        # preds_ = self.final_estimator.predict(preds)
        return preds_


In [147]:
vgr = MSBoostRegressor()

In [141]:
%%time
vgr.fit(X_train, y_train, complexity=True, n_random_models=24, n_estimators=100, stacking_model=ExtraTreesRegressor)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000069 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 750
[LightGBM] [Info] Number of data points in the train set: 562, number of used features: 4
[LightGBM] [Info] Start training from score 461.573951
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [143]:

print(f"Mean Squared Error on Validation Set: {mean_squared_error(vgr.predict(X_test), y_test)}")
print(f"Mean Squared Error on Train Set: {mean_squared_error(vgr.predict(X_train), y_train)}")

Mean Squared Error on Validation Set: 124.5495830247527
Mean Squared Error on Train Set: 1.2076523784024792e-16


In [59]:
"""
LassoLarsCV
Mean Squared Error on Validation Set: 171.01930457971707
Mean Squared Error on Train Set: 29.438274843939688

RidgeCV
Mean Squared Error on Validation Set: 148.71013233030047
Mean Squared Error on Train Set: 20.591860550481663

ElasticNetCV
Mean Squared Error on Validation Set: 273.7424629446183
Mean Squared Error on Train Set: 67.42811448485936

LarsCV
Mean Squared Error on Validation Set: 142.17259206898996
Mean Squared Error on Train Set: 21.997001681469357

LssoCV
Mean Squared Error on Validation Set: 296.1909887252867
Mean Squared Error on Train Set: 73.3420243616899

Mean Squared Error on Validation Set: 203.57414206312825
Mean Squared Error on Train Set: 39.41676735344899

LinearRegression
Mean Squared Error on Validation Set: 296.1909887252867
Mean Squared Error on Train Set: 73.3420243616899

LassoLarsIC
Mean Squared Error on Validation Set: 133.0058263527202
Mean Squared Error on Train Set: 28.106898244896545

ExtraTreesRegressor
Mean Squared Error on Validation Set: 114.65752126268903
Mean Squared Error on Train Set: 1.5758626100202367e-14

BaggingRegressor
Mean Squared Error on Validation Set: 160.78432081515896
Mean Squared Error on Train Set: 12.219357204084542
"""

'\nLassoLarsCV\nMean Squared Error on Validation Set: 171.01930457971707\nMean Squared Error on Train Set: 29.438274843939688\n\nRidgeCV\nMean Squared Error on Validation Set: 148.71013233030047\nMean Squared Error on Train Set: 20.591860550481663\n\nElasticNetCV\nMean Squared Error on Validation Set: 273.7424629446183\nMean Squared Error on Train Set: 67.42811448485936\n\nLarsCV\nMean Squared Error on Validation Set: 142.17259206898996\nMean Squared Error on Train Set: 21.997001681469357\n\nLssoCV\nMean Squared Error on Validation Set: 296.1909887252867\nMean Squared Error on Train Set: 73.3420243616899\n\nMean Squared Error on Validation Set: 203.57414206312825\nMean Squared Error on Train Set: 39.41676735344899\n\nLinearRegression\nMean Squared Error on Validation Set: 296.1909887252867\nMean Squared Error on Train Set: 73.3420243616899\n\nLassoLarsIC\nMean Squared Error on Validation Set: 133.0058263527202\nMean Squared Error on Train Set: 28.106898244896545\n\nExtraTreesRegressor\

In [144]:
from sklearn.datasets import fetch_openml

In [145]:
fetch_openml(data_id=379)

{'data':     id                                               text
 0    1  [1][hypercldb.gif] (update September 1995 ) [2...
 1    2  The Goat Shrine This page is fondly dedicated ...
 2    3  GRAND CHAMPION PYGMY GOAT NAMED Carol Shea, Je...
 3    5  Wild Goat Women Conference #2 After a very suc...
 4    6  Mountain Goats This is just some info I have f...
 ..  ..                                                ...
 65  70  Bean Central Aura_x Trust No One Call 1.800.JA...
 66  71  [bluff.gif] BIGHORN SHEEP AND INDUSTRIAL DEVEL...
 67  72  The News & Observer CAPTION: Wednesday, Oct. 1...
 68  73  Titles with initial 'p' Last update made at Fr...
 69  74  3A. ESSAY: IN ORDER FOR THE ADMISSIONS STAFF T...
 
 [70 rows x 2 columns],
 'target': 0     cold
 1      hot
 2      hot
 3      hot
 4     cold
       ... 
 65     hot
 66     hot
 67     hot
 68    cold
 69    cold
 Name: class, Length: 70, dtype: category
 Categories (3, object): ['cold', 'hot', 'medium'],
 'frame':     id      